In [1]:
import pandas as pd
import numpy as np
import boto3
import re

from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk

from s3 import get_file,upload_file

import string

In [2]:
s3 = boto3.resource('s3')
lyrics = get_file(s3,'s3ssp',download_file='NLP_Data/rough_lyrics_audio_features.csv',rename_file='0320_2020_nlp.csv')

In [3]:
df = pd.read_csv(lyrics,sep='|',encoding='utf-8')
df.head()
df = df.drop_duplicates(['artist_uri_x','track_uri'])
df_copy = df.copy().dropna()

#Running on this variable
df_sample = df_copy.head(10)

## Clean from Instrumentals

In [4]:
#Clean dataset of instrumentals
df_playlist = df_copy.copy()

df_playlist['instrumental'] = df_playlist['lyrics'].apply(lambda x: x.find('instrumental'))
df_playlist = df_playlist.drop(df_playlist[df_playlist['instrumental'] != -1].index)
#df = df.drop(df[df['lyrics']=='This is an instrumental'].index)

In [5]:
df_playlist['Instrumental'] = df_playlist['lyrics'].apply(lambda x: x.find('Instrumental'))
df_playlist =df_playlist.drop(df_playlist[df_playlist['Instrumental'] != -1].index)

In [6]:
df_playlist['INSTRUMENTAL'] = df_playlist['lyrics'].apply(lambda x: x.find('INSTRUMENTAL'))
df_playlist =df_playlist.drop(df_playlist[df_playlist['INSTRUMENTAL'] != -1].index)

In [7]:
df_playlist[df_playlist['lyrics'] == 'INSTRUMENTAL']

,artist_uri_x,artist_name,track_uri,track_name,lyrics,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,instrumental,Instrumental,INSTRUMENTAL


## Clean of stop words from NLTK and Github Song Stopwords Resource

In [8]:
def clean_stopwords(file,lyrics):
    df_copy['lyrics'] = lyrics_list.to_frame()
    lastfm_stopwords = df_lastfm_stopwords[0].str.replace('\n',' ')
    df_lastfm_stopwords['last_fm_stopwords'] = lastfm_stopwords.to_frame()
    df_lastfm_stopwords_list = df_lastfm_stopwords['last_fm_stopwords'].tolist()
    

In [9]:
df_lastfm_stopwords = pd.read_csv('stopwords.txt',header=None)
df_english = pd.read_csv("english.txt",header=None)

In [10]:
lyrics_list = df_copy['lyrics'].str.replace('\n',' ')
df_copy['lyrics'] = lyrics_list.to_frame()
lastfm_stopwords = df_lastfm_stopwords[0].str.replace('\n',' ')
df_lastfm_stopwords['last_fm_stopwords'] = lastfm_stopwords.to_frame()
#df_clean = .str.replace('(\[.*\])*','')
df_lastfm_stopwords_list = df_lastfm_stopwords['last_fm_stopwords'].tolist()

github_english_stopwords = df_english[0].str.replace('\n',' ')
df_english['github_english_stopwords'] = github_english_stopwords.to_frame()
github_english_stopwords_list = df_english['github_english_stopwords'].tolist()





In [11]:
df_lastfm_stopwords.head()

,0,last_fm_stopwords
0,a,a
1,able,able
2,about,about
3,above,above
4,abst,abst


In [12]:
df_lastfm_stopwords_list = df_lastfm_stopwords['last_fm_stopwords'].tolist()

In [13]:
df_spanish = pd.read_csv("spanish.txt",header=None)
github_spanish_stopwords = df_spanish[0].str.replace('\n',' ')
df_spanish['github_spanish_stopwords'] = github_spanish_stopwords.to_frame()
github_spanish_stopwords_list = df_spanish['github_spanish_stopwords'].tolist()

In [14]:
type(github_spanish_stopwords_list)

list

In [15]:
import spacy
regexp = re.compile('(?u)\\b\\w\\w+\\b')
en_nlp = spacy.load('en_core_web_sm',disable=['parser','ner'])
old_tokenizer = en_nlp.tokenizer
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(regexp.findall(string))

In [16]:
def d_preprocessor(lyric):
        doc_spacy = en_nlp(lyric)
        lemma_lyric = [token.lemma_ for token in doc_spacy]
        lemma = ' '.join(lemma_lyric)
        return lemma.lower()

In [17]:
df_sample['lemma'] = df_sample['lyrics'].apply(lambda x: d_preprocessor(x))

/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_sample[['lyrics','lemma']]

,lyrics,lemma
0,"N-n-now, honey You better sit down and look ar...",now honey -pron- better sit down and look arou...
1,I lay alone awake at night Sorrow fills my eye...,lay alone awake at night sorrow fill -pron- ey...
2,Youre everything I thought you never were And ...,youre everything think -pron- never be and not...
3,He is always laughin' and flirting with me And...,-pron- be always laughin and flirt with -pron-...
4,"This is for my fans (Uhu, uhu) This is for my ...",this be for -pron- fan uhu uhu this be for -pr...
6,[Verse 1: Beyonce] I've been watching for the ...,verse beyonce ve be watch for the sign take tr...
7,[Verse 1] When the palm of my two hands hold e...,verse when the palm of -pron- two hand hold ea...
8,[Intro: Presenter] The winner is Beyonce Knowl...,intro presenter the winner be beyonce knowles ...
10,[Intro] Your challengers are a young group fro...,intro -pron- challenger be young group from ho...
11,"Part 1: ""YoncÃ©"" [Intro] Let me hear you say ""...",part yoncã intro let -pron- hear -pron- say he...


## Clean stopwords and punctuation

In [19]:
def preprocessor(text,source):
    tokens = word_tokenize(text)
    return (" ").join([word for word in tokens if word not in source and word != '-pron-' and word.isalpha()])
            
    #return (" ").join([word for word in if word.lower() not in source])

In [20]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adamgoldstein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamgoldstein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Add German Words and Two Letter Words and other stopwords neccessary

In [21]:
df_demo = df_sample

df_demo['git_english_lyrics'] = df_demo['lemma'].apply(lambda x: preprocessor(x,github_english_stopwords_list))
df_demo['git_spanish_lyrics'] = df_demo['git_english_lyrics'].apply(lambda x: preprocessor(x,github_spanish_stopwords_list))
df_demo['last_fm_lyrics'] = df_demo['git_spanish_lyrics'].apply(lambda x: preprocessor(x,df_lastfm_stopwords_list))
df_demo['nltk_lyrics'] = df_demo['last_fm_lyrics'].apply(lambda x: preprocessor(x,stopwords.words('english')))

#df_demo['nltk_lyrics'] = df_demo['git_spanish_lyrics'].apply(lambda x: preprocessor(x,stopwords.words('english')))


/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is tr

## Clean Lyrics Dataset

In [22]:
df_topic = df_demo[['artist_name','track_name','track_uri','acousticness','danceability','energy','valence'
                    ,'instrumentalness','key','mode','liveness','speechiness','time_signature',
                    'tempo','loudness','nltk_lyrics']]
df_topic = df_topic.rename(columns={'nltk_lyrics':'lyrics'})

In [23]:
df_topic['lyrics'].values

array(['honey sit bump head love talk sense baby hate kid dog check credential friend woman love baby love damn easy love baby damn easy beauty class style ass care care looka money bank account notice love baby love damn easy love baby damn easy beauty heart head book sharp care smart bedroom happy nasty tune ooh love baby love damn easy love baby damn easy love love lovely plain dumb',
       'lay awake night sorrow eye strong disguise leave shoulder lean guess soldier gon save hero leave save hero save girl save save bottle hurt guess living lie mind day die bring life simple word gesture beautiful bury treasure eye lead pot gold save hero leave save hero save girl save save drive crazy save save save hero leave save hero save girl save save',
       'couldve live forget love forgive break heart time hate erase time hurt tear hate pain day wan babe break heart wan breath babe wan play love love break heart wan play break hearte girlno broken hearted girl broken hearted girl feel afr

In [24]:
#df_topic[(df_topic['artist_name']=='Arcade Fire') & (df_topic['track_name']=='here comes the night time')]

In [25]:
#df_topic.to_csv('master_lyrics_audio_features.csv',index=False,sep='|')

## Upload file to S3

In [26]:
#upload_file(s3,'s3ssp',upload_file='master_lyrics_audio_features.csv',rename_file='NLP_Data/master_lyrics_audio_features.csv')